# Korean Embedding 모델을 SageMaker 배포 및 추론
- 이 노트북은 SageMaker Notebook Instance 의 conday_pytorch_p39 에서 테스트 되었습니다. 

Model Ref:
- BM-K/KoSimCSE-roberta
    - https://huggingface.co/BM-K/KoSimCSE-roberta
Inference Code Ref:    
- Huggingface Sagemaker-sdk - Deploy 🤗 Transformers for inference
    - https://github.com/huggingface/notebooks/blob/main/sagemaker/11_deploy_model_from_hf_hub/deploy_transformer_model_from_hf_hub.ipynb
- Sentence Embeddings with Hugging Face Transformers, Sentence Transformers and Amazon SageMaker - Custom Inference for creating document embeddings with Hugging Face's Transformers
    - https://github.com/huggingface/notebooks/blob/main/sagemaker/17_custom_inference_script/sagemaker-notebook.ipynb
    

# 0. 기본 환경 설정

In [2]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('../common_code')

# 1. HF Hub로 부터 모델 및 토큰 나이저 로딩

In [3]:
import torch
from transformers import AutoModel, AutoTokenizer


model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta')
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta')


In [4]:
sample = "이번 주 일요일에 분당 이마트 점은 문을 여나요"

inputs = tokenizer(sample, padding=True, truncation=True, return_tensors="pt")
embeddings, _ = model(**inputs, return_dict=False)

emb_len = len(embeddings[0][0])
print("sample : \n", sample)
print("embeding size: ", emb_len)
print(f"embeding content from 0 to 10 out of {emb_len}: \n", embeddings[0][0][0:10])

sample : 
 이번 주 일요일에 분당 이마트 점은 문을 여나요
embeding size:  768
embeding content from 0 to 10 out of 768: 
 tensor([-0.2569, -0.1982,  0.8970, -1.7043, -0.1197,  0.2873,  0.3933, -0.4806,
        -0.1716, -0.6642], grad_fn=<SliceBackward0>)


## 추론 테스트 및 문장 유사도 측정
- 아래 첫문장, 두번째 문장의 유사도를 구함
- 아래 첫문장, 세째 문장의 유사도를 구함
- 최종적으로 유사도 수치를 비교 함

In [5]:
def show_embedding_score(tokenizer, model, sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    embeddings, _ = model(**inputs, return_dict=False)

    score01 = cal_score(embeddings[0][0], embeddings[1][0])
    score02 = cal_score(embeddings[0][0], embeddings[2][0])

    print(score01, score02)

def cal_score(a, b):
    '''
    코사인 유사도 구하는 함수
    '''
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100
    

In [6]:
sentences1 = ['이번 주 일요일에 분당 이마트 점은 문을 여나요',
             '일요일에 분당 이마트는 문 열어요?',
             '분당 이마트 점은 토요일에 몇 시까지 하나요']

show_embedding_score(tokenizer, model, sentences1)    

tensor([[92.7287]], grad_fn=<MulBackward0>) tensor([[79.8030]], grad_fn=<MulBackward0>)


# 2. 세이지 메이커로 모델 배포

In [7]:
import sagemaker
import boto3

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

print(f"sagemaker role arn: {role}")

sagemaker role arn: arn:aws:iam::057716757052:role/gen_ai_gsmoon


## HF Model ID, HF_TASK 정의

In [8]:
from sagemaker.huggingface import HuggingFaceModel

# Hub Model configuration. https://huggingface.co/models
hub = {
  'HF_MODEL_ID':'BM-K/KoSimCSE-roberta', # model_id from hf.co/models
  'HF_TASK':'feature-extraction',
  'SAGEMAKER_MODEL_SERVER_TIMEOUT':'3600', 
  'TS_MAX_RESPONSE_SIZE':'2000000000',
  'TS_MAX_REQUEST_SIZE':'2000000000',
  'MMS_MAX_RESPONSE_SIZE':'2000000000',
  'MMS_MAX_REQUEST_SIZE':'2000000000'
}        

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.26", # transformers version used
   pytorch_version="1.13", # pytorch version used
   py_version="py39", # python version of the DLC
)

## 모델 배포

In [9]:
from datetime import datetime

time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

endpoint_name = f"KoSimCSE-roberta-" + time_stamp

In [10]:
%%time

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   endpoint_name = endpoint_name,
   instance_type="ml.m5.xlarge"
)

----!CPU times: user 157 ms, sys: 5.86 ms, total: 162 ms
Wall time: 2min 32s


# 3. 추론

In [11]:
sentences2 = ['분당 이마트점에 KT 대리점이 있나요?',
             '거기 이마트점에 KT 대리점이 있나요?',
             '분당 아미트 점은 지하 주차장이 있나요?']


In [12]:
import numpy as np

payload_1 = {
    "inputs" : sentences1
}

payload_2 = {
    "inputs" : sentences2
}

def predict_payload(data):
    res = predictor.predict(data=data)
    res = np.array(res) # .squeeze().squeeze()
    # print("res: ", res.shape)
    # print("embedding dimension: ", len(res[0][0][0]))
    return res



## Sample Test (한개의 문장 임베딩 보여 주기)

In [13]:
payload_0 = {
    "inputs" : "이번 주 일요일에 분당 이마트 점은 문을 여나요"
}


response = predict_payload(payload_0)
emb_len = len(response[0][0])
print("payload_0 : \n", payload_0)
print("embeding size: ", emb_len)
print(f"embeding content from 0 to 10 out of {emb_len}: ", response[0][0][0:10])

payload_0 : 
 {'inputs': '이번 주 일요일에 분당 이마트 점은 문을 여나요'}
embeding size:  768
embeding content from 0 to 10 out of 768:  [-0.2569178  -0.19823857  0.89699048 -1.70428038 -0.11973442  0.28725004
  0.39328021 -0.48056296 -0.17160198 -0.66424179]


In [14]:
def show_embedding_score2(payload):
    '''
    # res 
    # 1st dim: samples, 2nd dim: place_hoder, 3rd_dim : CLS, ohter tokens 
    # res.shape --> (3,1)
    # len(res[1][0]) --> 11 두번째 샘플의 11개 토큰
    # len(res[1][0][0]) --> 두번째 샘플의 , 첫번째 토큰 임베딩 (764 사이즈)
    '''
    res = predict_payload(payload)    
    embeddings_0 = torch.Tensor(res[0][0][0]) 
    embeddings_1 = torch.Tensor(res[1][0][0])
    embeddings_2 = torch.Tensor(res[2][0][0])

    score01 = cal_score(embeddings_0, embeddings_1)
    score02 = cal_score(embeddings_0, embeddings_2)    
    print(score01, score02)
    


In [15]:
print("payload_1: \n", payload_1)
show_embedding_score2(payload_1)

payload_1: 
 {'inputs': ['이번 주 일요일에 분당 이마트 점은 문을 여나요', '일요일에 분당 이마트는 문 열어요?', '분당 이마트 점은 토요일에 몇 시까지 하나요']}
tensor([[92.7287]]) tensor([[79.8030]])


<ipython-input-12-c7c7080415fb>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res = np.array(res) # .squeeze().squeeze()


In [16]:
print("payload_2: \n", payload_2)
show_embedding_score2(payload_2)

payload_2: 
 {'inputs': ['분당 이마트점에 KT 대리점이 있나요?', '거기 이마트점에 KT 대리점이 있나요?', '분당 아미트 점은 지하 주차장이 있나요?']}
tensor([[89.2611]]) tensor([[53.1729]])


<ipython-input-12-c7c7080415fb>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res = np.array(res) # .squeeze().squeeze()


# 4. Boto3 invoke_endpoint() 사용하여 추론

In [17]:
endpoint_name = predictor.endpoint_name

In [18]:
import boto3
import json

def query_endpoint_embedding_with_json_payload(encoded_json, endpoint_name, content_type="application/json"):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType=content_type, Body=encoded_json
    )
    return response

def transform_output(output: bytes) -> str:
    response_json = json.loads(output.read().decode("utf-8"))
    # return response_json
    return response_json[0][0]


In [19]:
sentences2_1 = "분당 이마트점에 KT 대리점이 있나요?"
sentences2_2 = "거기 이마트점에 KT 대리점이 있나요?"

payload_2_1 = {
    "inputs" : sentences2_1
}

payload_2_2 = {
    "inputs" : sentences2_2
}

# 첫번째 문장
query_response = query_endpoint_embedding_with_json_payload(
    json.dumps(payload_2_1).encode("utf-8"), endpoint_name=endpoint_name
)

emb_1 = transform_output(query_response['Body'])
print("첫문장 임베딩 사이즈: ", len(emb_1))

# 두번째 문장
query_response = query_endpoint_embedding_with_json_payload(
    json.dumps(payload_2_2).encode("utf-8"), endpoint_name=endpoint_name
)
 
emb_2 = transform_output(query_response['Body'])
print("두번째 문장 임베딩 사이즈: ", len(emb_2))

첫문장 임베딩 사이즈:  768
두번째 문장 임베딩 사이즈:  768


In [20]:
def show_embedding_score3(emb1, emb2):

    embeddings_0 = torch.Tensor(emb1) 
    embeddings_1 = torch.Tensor(emb2)

    score01 = cal_score(embeddings_0, embeddings_1)

    print(score01)

show_embedding_score3(emb_1, emb_2)    

tensor([[89.2611]])


# 5. 엔드포인트 삭제

In [21]:
# # delete endpoint
# predictor.delete_model()
# predictor.delete_endpoint()

In [22]:
embedding_model_endpoint_name = endpoint_name
%store embedding_model_endpoint_name

Stored 'embedding_model_endpoint_name' (str)


In [23]:
embedding_model_endpoint_name

'KoSimCSE-roberta-2023-06-04-11-24-26'